# Market Data Integration Guide

## Overview
This guide covers integrating with TastyTrade's market data API using Python WebSocket implementation, with a focus on asyncio-based approach.

## API Resources
- [Developer Portal](https://developer.tastytrade.com/)
- [Streaming Market Data Documentation](https://developer.tastytrade.com/streaming-market-data/)
- [DXLink Protocol Reference](https://demo.dxfeed.com/dxlink-ws/debug/#/protocol)

## References
- dxFeed API [doc](https://dxfeed.readthedocs.io/en/stable/) | [github](https://github.com/dxFeed/dxfeed-python-api) | [pypi](https://pypi.org/project/dxfeed/)
- Web Widget Demo [docs](https://tools.dxfeed.com/python-demo/?_ga=2.236532102.669062738.1733058631-563197392.1733058631) | [github](https://github.com/dxFeed/dxfeed-python-api-web-widget-example?tab=readme-ov-file)

## WebSocket Implementation Details

### Selected Library
We'll be using the [websockets](https://websockets.readthedocs.io/en/stable/) library for Python, which provides high-level WebSocket implementation (not to be confused with the lower-level `websocket` library).

### Key Resources
- **Documentation**: Complete guides available on [readthedocs.io](https://websockets.readthedocs.io/en/stable/)
- **Reference Implementation**: [Example using Threading](https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py) by [LordKaT](https://github.com/LordKaT)

### Technical Approach
The implementation will use `asyncio` due to its comprehensive [feature support](https://websockets.readthedocs.io/en/stable/reference/features.html) in the websockets library.

### AsyncIO Learning Resources
For better understanding of the asyncio implementation:
- [How does async/await work in Python 3.5](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/)
- [Cooperative multitasking with Coroutines](https://pymotw.com/3/asyncio/coroutines.html)
- [A Curious course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/)


In [1]:
from tastytrade.logging import setup_logging
import logging
import asyncio
from tastytrade.sessions import Credentials
from tastytrade.sessions.requests import AsyncSessionHandler
from tastytrade.sessions.sockets import WebSocketManager
from tastytrade.sessions.dxlink import DXLinkClient
from decimal import Decimal
from datetime import datetime, timedelta

import polars as pl

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

shutdown = asyncio.Event()


setup_logging(
    level=logging.INFO,
    # log_dir="/var/log/tastytrade",
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)


def get_trade_day() -> str:
    trade_day = datetime.now()
    while trade_day.weekday() >= 5:
        trade_day += timedelta(days=1)

    return trade_day.strftime("%y%m%d")


def round_decimal(value, precision=5) -> float:
    target = str(precision)

    if not isinstance(value, Decimal):
        value = Decimal(value)

    if precision > 0:
        result = (value / Decimal(target)).quantize(
            Decimal("1"), rounding="ROUND_HALF_UP"
        ) * Decimal(target)
        return float(result)

    else:
        return float(round(value, abs(precision)))

2025-01-13 14:46:05 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250113.log


## Test WebSocket context manager

In [2]:
"""
TODO 

  1. Find nearest ATM strike
  2. list of symbols
      +-10 pts from ATM strike
      both Calls & Puts
"""

session = await AsyncSessionHandler.create(Credentials(env=ENV))

yy_mm_dd = get_trade_day()

symbols = [
    "SPX",
    "NVDA",
    "BTC/USD:CXTALP",
    "BCH/USD:CXTALP",
    "ETH/USD:CXTALP",
    f".SPXW{yy_mm_dd}C5915",
    f".SPXW{yy_mm_dd}C5910",
    f".SPXW{yy_mm_dd}P5910",
    f".SPXW{yy_mm_dd}P5905",
]

async with WebSocketManager(session) as websocket:
    dxlink_client = DXLinkClient(websocket)
    await dxlink_client.setup_feeds()
    await dxlink_client.subscribe_to_feeds(symbols)

    if TEST:
        await asyncio.sleep(DURATION)
    else:
        await websocket.send_keepalives()

    await dxlink_client.queue_manager.cleanup()

await session.close()

2025-01-13 14:46:09 - INFO:tastytrade.sessions.requests:144:Session created successfully
2025-01-13 14:46:09 - INFO:tastytrade.sessions.messaging:64:Started Channels.Control listener on channel 0
2025-01-13 14:46:09 - INFO:tastytrade.sessions.messaging:64:Started Channels.Quotes listener on channel 3
2025-01-13 14:46:09 - INFO:tastytrade.sessions.messaging:64:Started Channels.Trades listener on channel 1
2025-01-13 14:46:09 - INFO:tastytrade.sessions.messaging:64:Started Channels.Greeks listener on channel 5
2025-01-13 14:46:09 - INFO:tastytrade.sessions.messaging:64:Started Channels.Profile listener on channel 7
2025-01-13 14:46:09 - INFO:tastytrade.sessions.messaging:64:Started Channels.Summary listener on channel 9
2025-01-13 14:46:09 - INFO:tastytrade.sessions.messaging:190:SETUP
2025-01-13 14:46:09 - INFO:tastytrade.sessions.messaging:193:AUTH_STATE:UNAUTHORIZED
2025-01-13 14:46:09 - INFO:tastytrade.sessions.messaging:193:AUTH_STATE:AUTHORIZED
2025-01-13 14:46:09 - INFO:tastytrade

In [3]:
trades  = dxlink_client.queue_manager.handlers["Trades"].processors["feed"].df

spx_trades = trades.filter(pl.col("eventSymbol") == "SPX")

spx_price = spx_trades.sort("timestamp")[-1].select("price").item()

In [4]:
dxlink_client.queue_manager.handlers["Quotes"].processors["feed"].df

eventSymbol,timestamp,bidPrice,askPrice,bidSize,askSize
str,datetime[μs],"decimal[*,2]","decimal[*,2]","decimal[*,1]","decimal[*,1]"
"""NVDA""",2025-01-13 19:46:09.978453,132.46,132.47,1010.0,112.0
"""BCH/USD:CXTALP""",2025-01-13 19:46:09.978462,412.77,415.75,9.9,10.0
""".SPXW250113C5915""",2025-01-13 19:46:09.978467,0.00,0.05,null,2028.0
""".SPXW250113C5910""",2025-01-13 19:46:09.978471,0.00,0.05,null,1681.0
""".SPXW250113P5910""",2025-01-13 19:46:09.978476,81.80,82.40,1.0,3.0
…,…,…,…,…,…
"""ETH/USD:CXTALP""",2025-01-13 19:46:24.163434,3024.58,3045.90,10.0,10.0
"""NVDA""",2025-01-13 19:46:24.507365,132.46,132.47,543.0,474.0
"""BCH/USD:CXTALP""",2025-01-13 19:46:24.633106,413.09,416.05,9.9,10.0


In [5]:
dxlink_client.queue_manager.handlers["Greeks"].processors["feed"].df

eventSymbol,timestamp,volatility,delta,gamma,theta,rho,vega
str,datetime[μs],"decimal[*,15]","decimal[*,19]","decimal[*,20]","decimal[*,16]","decimal[*,21]","decimal[*,18]"
""".SPXW250113C5915""",2025-01-13 19:46:09.979920,0.301642704957473,0.0001167923003537506,0.00001987068933944243,-0.0553451356099034,0.000001168353850718037,0.000349899868615896
""".SPXW250113C5910""",2025-01-13 19:46:09.979929,0.301642704957473,0.0002644276764468730,0.00004267021692204860,-0.0567149925730407,0.000002645128669533070,0.000751373192937026
""".SPXW250113P5910""",2025-01-13 19:46:09.979936,0.301757568692550,-0.9996314979593910000,0.00004284961808852840,-0.0569965692307761,-0.010147749411009510000,0.000754819561446821
""".SPXW250113P5905""",2025-01-13 19:46:09.979943,0.301757568692550,-0.9993209336879850000,0.00008790280318289280,-0.0583712098951850,-0.010136055579340550000,0.001548456166199078
""".SPXW250113C5910""",2025-01-13 19:46:11.960556,0.333528746039614,0.0006699250482527230,0.00009192523982548510,-0.0540799638441223,0.000006624356706008400,0.001769069535008773
""".SPXW250113C5915""",2025-01-13 19:46:12.060209,0.333528746039614,0.0003346538192917860,0.00004833122427858460,-0.0529322900190707,0.000003309298648460480,0.000930117741582615
""".SPXW250113P5905""",2025-01-13 19:46:12.060229,0.333520142912005,-0.9986268265368190000,0.00016839486444173550,-0.0552318620975996,-0.010016770566052490000,0.003240617494537260
""".SPXW250113P5910""",2025-01-13 19:46:12.060245,0.333520142912005,-0.9992518953279620000,0.00009190318849737910,-0.0540620005652528,-0.010031443088532840000,0.001768599543909540


In [6]:
dxlink_client.queue_manager.handlers["Summary"].processors["feed"].df

eventSymbol,timestamp,openInterest,dayOpenPrice,dayHighPrice,dayLowPrice,prevDayClosePrice
str,datetime[μs],"decimal[*,0]","decimal[*,2]","decimal[*,2]","decimal[*,2]","decimal[*,2]"
"""SPX""",2025-01-13 19:46:09.981098,0,5782.02,5830.76,5773.31,5827.04
"""NVDA""",2025-01-13 19:46:09.981109,0,129.99,133.11,129.51,135.91
""".SPXW250113C5915""",2025-01-13 19:46:09.981114,1661,0.12,0.15,0.05,0.80
""".SPXW250113C5910""",2025-01-13 19:46:09.981119,1236,0.15,0.20,0.05,1.15
""".SPXW250113P5910""",2025-01-13 19:46:09.981124,383,128.92,134.07,88.12,89.12
""".SPXW250113P5905""",2025-01-13 19:46:09.981129,649,125.04,128.51,84.93,83.62
"""BCH/USD:CXTALP""",2025-01-13 19:46:09.987954,0,434.51,435.22,400.25,434.50
"""BTC/USD:CXTALP""",2025-01-13 19:46:09.987962,0,94116.58,94292.23,89261.26,94116.52
"""ETH/USD:CXTALP""",2025-01-13 19:46:09.987968,0,3223.68,3232.28,2879.90,3223.67


In [7]:
dxlink_client.queue_manager.handlers["Profile"].processors["feed"].df

eventSymbol,timestamp,description,shortSaleRestriction,tradingStatus,statusReason,haltStartTime,haltEndTime,highLimitPrice,lowLimitPrice,high52WeekPrice,low52WeekPrice
str,datetime[μs],str,str,str,str,i64,i64,"decimal[*,0]","decimal[*,0]","decimal[*,2]","decimal[*,2]"
"""SPX""",2025-01-13 19:46:09.984560,"""S&P 500 INDEX""","""UNDEFINED""","""UNDEFINED""",null,0,0,null,null,6099.97,4714.82
"""BCH/USD:CXTALP""",2025-01-13 19:46:09.984570,"""Bitcoin Cash""","""UNDEFINED""","""ACTIVE""",null,0,0,null,null,718.86,218.44
""".SPXW250113C5915""",2025-01-13 19:46:09.986857,"""""","""UNDEFINED""","""UNDEFINED""",null,0,0,null,null,null,null
""".SPXW250113P5905""",2025-01-13 19:46:09.986867,"""""","""UNDEFINED""","""UNDEFINED""",null,0,0,null,null,null,null
"""BTC/USD:CXTALP""",2025-01-13 19:46:09.986879,"""Bitcoin""","""UNDEFINED""","""ACTIVE""",null,0,0,null,null,107999.36,39354.48
""".SPXW250113P5910""",2025-01-13 19:46:09.986884,"""""","""UNDEFINED""","""UNDEFINED""",null,0,0,null,null,null,null
""".SPXW250113C5910""",2025-01-13 19:46:09.986888,"""""","""UNDEFINED""","""UNDEFINED""",null,0,0,null,null,null,null
"""ETH/USD:CXTALP""",2025-01-13 19:46:09.986893,"""Ethereum""","""UNDEFINED""","""ACTIVE""",null,0,0,null,null,4109.98,2112.57
"""NVDA""",2025-01-13 19:46:10.008551,"""NVIDIA Corp""","""UNDEFINED""","""ACTIVE""","""Trading on NASDAQ: Reason Not …",0,0,null,null,153.13,54.74


In [8]:
import sys

sys.exit("Done")

SystemExit: Done

/Users/xaviermandeng/.pyenv/versions/tasty/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Test individual components

In [ ]:
session = await AsyncSessionHandler.create(Credentials(env=ENV))

exlink = WebSocketManager(session)
await exlink.open()

In [ ]:
dxlink_client = DXLinkClient(exlink)
await dxlink_client.setup_feeds()
await dxlink_client.subscribe_to_feeds(["SPX"])

In [ ]:
asyncio.all_tasks()

In [ ]:
await exlink.close()

In [ ]:
dxlink_client.session.close()

In [ ]:
dxlink_client

In [16]:
symbol = "SPX"

response = session.session.get(session.base_url + "/option-chains/" + symbol)

In [ ]:
from tastytrade.markets.instruments import get_option_chains
import polars as pl

# Using your existing session
df = await get_option_chains(session, "SPX")
print(df)

In [ ]:

df = pl.DataFrame(response["data"]["items"])
df.head()

In [5]:
from enum import Enum


class OptionType(Enum):
    CALL = "C"
    PUT = "P"


date = get_trade_day()
trade_date = f"20{date[:2]}-{date[2:4]}-{date[4:]}"

In [ ]:
(
    df.with_columns(pl.col("strike-price").cast(pl.Float32(), strict=False))
    .with_columns(
        pl.col("option-type").map_elements(lambda x: OptionType(x).name, return_dtype=str)
    )
    .filter((pl.col("expiration-date") == trade_date))
    .filter((pl.col("strike-price") >= 5700))
    .select(["strike-price", "option-type", "streamer-symbol"])
    .sort("strike-price")
    .head(10)
)